<a href="https://colab.research.google.com/github/Putranto13/Tugas_Machine_Learning/blob/main/FINAL%20Exam/Exam_Paper1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
data = '/content/MnistSimpleCNN-master.zip' #mengambil data yang telah di upload di google colab dan mengekstraknya
!unzip MnistSimpleCNN-master.zip

Archive:  MnistSimpleCNN-master.zip
3a13cdb79c633c96384318f6c2b634179ce3d191
   creating: MnistSimpleCNN-master/
  inflating: MnistSimpleCNN-master/README.md  
   creating: MnistSimpleCNN-master/code/
   creating: MnistSimpleCNN-master/code/__pycache__/
  inflating: MnistSimpleCNN-master/code/__pycache__/augment.cpython-37.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/datasets.cpython-37.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/datasets.cpython-38.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/ema.cpython-37.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/ema.cpython-38.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/mnist.cpython-37.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/models.cpython-37.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/models.cpython-38.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/transforms.cpython-37.pyc  
  inflating: MnistSimpleCNN-master/code/__pycache__/transfor

In [ ]:
class EMA:
    def __init__(self, model, decay):
        self.decay = decay
        self.shadow = {}
        self.original = {}

        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def __call__(self, model, num_updates):
        decay = min(self.decay, (1.0 + num_updates) / (10.0 + num_updates))
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - decay) * param.data + decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def assign(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.original[name] = param.data.clone()
                param.data = self.shadow[name]

    def resume(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                param.data = self.original[name]



In [ ]:
#mengimport library
import torch
import numpy as np
from PIL import Image
from torchvision import transforms

class MnistDataset(torch.utils.data.Dataset):
    def __init__(self, training=True, transform=None):
        if training==True:
            f = open('/content/MnistSimpleCNN-master/data/MNIST/raw/train-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('/content/MnistSimpleCNN-master/data/MNIST/raw/train-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        else:
            f = open('/content/MnistSimpleCNN-master/data/MNIST/raw/t10k-images-idx3-ubyte', 'rb')
            xs = np.array(np.frombuffer(f.read(), np.uint8, offset=16))
            f.close()
            f = open('/content/MnistSimpleCNN-master/data/MNIST/raw/t10k-labels-idx1-ubyte', 'rb')
            ys = np.array(np.frombuffer(f.read(), np.uint8, offset=8))
            f.close()
        xs = np.reshape(xs, (-1, 28, 28, 1)).astype(np.float32)
        ys = ys.astype(np.int)
        self.x_data = xs
        self.y_data = ys
        self.transform = transform

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = Image.fromarray(self.x_data[idx].reshape(28, 28))
        y = torch.tensor(np.array(self.y_data[idx]))
        if self.transform:
            x = self.transform(x)
        x = transforms.ToTensor()(np.array(x)/255)
        return x, y



In [ ]:
import random
import torchvision.transforms.functional as W

class RandomRotation(object):
    def __init__(self, degrees, seed=1):
        self.degrees = (-degrees, degrees)
        random.seed(seed)
    
    @staticmethod
    def get_params(degrees):
        angle = random.uniform(degrees[0], degrees[1])
        return angle

    def __call__(self, img):
        angle = self.get_params(self.degrees)
        return W.rotate(img, angle, False, False, None, None)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM3(nn.Module):
    def __init__(self):
        super(ModelM3, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, bias=False)       # output becomes 26x26
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 48, 3, bias=False)      # output becomes 24x24
        self.conv2_bn = nn.BatchNorm2d(48)
        self.conv3 = nn.Conv2d(48, 64, 3, bias=False)      # output becomes 22x22
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 80, 3, bias=False)      # output becomes 20x20
        self.conv4_bn = nn.BatchNorm2d(80)
        self.conv5 = nn.Conv2d(80, 96, 3, bias=False)      # output becomes 18x18
        self.conv5_bn = nn.BatchNorm2d(96)
        self.conv6 = nn.Conv2d(96, 112, 3, bias=False)     # output becomes 16x16
        self.conv6_bn = nn.BatchNorm2d(112)
        self.conv7 = nn.Conv2d(112, 128, 3, bias=False)    # output becomes 14x14
        self.conv7_bn = nn.BatchNorm2d(128)
        self.conv8 = nn.Conv2d(128, 144, 3, bias=False)    # output becomes 12x12
        self.conv8_bn = nn.BatchNorm2d(144)
        self.conv9 = nn.Conv2d(144, 160, 3, bias=False)    # output becomes 10x10
        self.conv9_bn = nn.BatchNorm2d(160)
        self.conv10 = nn.Conv2d(160, 176, 3, bias=False)   # output becomes 8x8
        self.conv10_bn = nn.BatchNorm2d(176)
        self.fc1 = nn.Linear(11264, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        conv6 = F.relu(self.conv6_bn(self.conv6(conv5)))
        conv7 = F.relu(self.conv7_bn(self.conv7(conv6)))
        conv8 = F.relu(self.conv8_bn(self.conv8(conv7)))
        conv9 = F.relu(self.conv9_bn(self.conv9(conv8)))
        conv10 = F.relu(self.conv10_bn(self.conv10(conv9)))
        flat1 = torch.flatten(conv10.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM5(nn.Module):
    def __init__(self):
        super(ModelM5, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, bias=False)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5, bias=False)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 96, 5, bias=False)
        self.conv3_bn = nn.BatchNorm2d(96)
        self.conv4 = nn.Conv2d(96, 128, 5, bias=False)
        self.conv4_bn = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 160, 5, bias=False)
        self.conv5_bn = nn.BatchNorm2d(160)
        self.fc1 = nn.Linear(10240, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        conv5 = F.relu(self.conv5_bn(self.conv5(conv4)))
        flat5 = torch.flatten(conv5.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat5))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ModelM7(nn.Module):
    def __init__(self):
        super(ModelM7, self).__init__()
        self.conv1 = nn.Conv2d(1, 48, 7, bias=False)    # output becomes 22x22
        self.conv1_bn = nn.BatchNorm2d(48)
        self.conv2 = nn.Conv2d(48, 96, 7, bias=False)   # output becomes 16x16
        self.conv2_bn = nn.BatchNorm2d(96)
        self.conv3 = nn.Conv2d(96, 144, 7, bias=False)  # output becomes 10x10
        self.conv3_bn = nn.BatchNorm2d(144)
        self.conv4 = nn.Conv2d(144, 192, 7, bias=False) # output becomes 4x4
        self.conv4_bn = nn.BatchNorm2d(192)
        self.fc1 = nn.Linear(3072, 10, bias=False)
        self.fc1_bn = nn.BatchNorm1d(10)
    def get_logits(self, x):
        x = (x - 0.5) * 2.0
        conv1 = F.relu(self.conv1_bn(self.conv1(x)))
        conv2 = F.relu(self.conv2_bn(self.conv2(conv1)))
        conv3 = F.relu(self.conv3_bn(self.conv3(conv2)))
        conv4 = F.relu(self.conv4_bn(self.conv4(conv3)))
        flat1 = torch.flatten(conv4.permute(0, 2, 3, 1), 1)
        logits = self.fc1_bn(self.fc1(flat1))
        return logits
    def forward(self, x):
        logits = self.get_logits(x)
        return F.log_softmax(logits, dim=1)


In [ ]:
# Mengimports library
import sys
import os
import argparse
import numpy as np 
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim #digunakan untuk optimasi
from torchvision import datasets, transforms
from torchsummary import summary
from PIL import Image

#Menjalankan Proses Training
def run(p_seed=0, p_epochs=150, p_kernel_size=5, p_logdir="temp"):
    # random number generator seed 
    SEED = p_seed
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)

    # kernel size model
    KERNEL_SIZE = p_kernel_size

    # Nomor Epoch
    NUM_EPOCHS = p_epochs

    
    if not os.path.exists("/content/MnistSimpleCNN-master/logs/%s"%p_logdir):
        os.makedirs("/content/MnistSimpleCNN-master/logs/%s"%p_logdir)
    OUTPUT_FILE = str("/content/MnistSimpleCNN-master/logs/%s/log%03d.out"%(p_logdir,SEED))
    MODEL_FILE = str("/content/MnistSimpleCNN-master/logs/%s/model%03d.pth"%(p_logdir,SEED))

    #Menjalankan GPU
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda == False:
        print("WARNING: CPU will be used for training.")
        exit(0)

    # data augmentation methods 
    transform = transforms.Compose([
        RandomRotation(20, seed=SEED),
        transforms.RandomAffine(0, translate=(0.2, 0.2)),
        ])

    # data loader 
    train_dataset = MnistDataset(training=True, transform=transform)
    test_dataset = MnistDataset(training=False, transform=None)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=120, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # Seleksi Model
    if(KERNEL_SIZE == 3):
        model = ModelM3().to(device)
    elif(KERNEL_SIZE == 5):
        model = ModelM5().to(device)
    elif(KERNEL_SIZE == 7):
        model = ModelM7().to(device)

    summary(model, (1, 28, 28))

    # hyperparameter selection 
    ema = EMA(model, decay=0.999)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

    f = open(OUTPUT_FILE, 'w')
    f.close()

    # Variable global 
    g_step = 0
    max_correct = 0

    # Pengulangan proses training dan evaluasi 
    for epoch in range(NUM_EPOCHS):
        # Proses Training                                                           #
        model.train()
        train_loss = 0
        train_corr = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            train_pred = output.argmax(dim=1, keepdim=True)
            train_corr += train_pred.eq(target.view_as(train_pred)).sum().item()
            train_loss += F.nll_loss(output, target, reduction='sum').item()
            loss.backward()
            optimizer.step()
            g_step += 1
            ema(model, g_step)
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{:05d}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))
        train_loss /= len(train_loader.dataset)
        train_accuracy = 100 * train_corr / len(train_loader.dataset)

       
        # Proses Test                                                             
       
        model.eval()
        ema.assign(model)
        test_loss = 0
        correct = 0
        total_pred = np.zeros(0)
        total_target = np.zeros(0)
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                total_pred = np.append(total_pred, pred.cpu().numpy())
                total_target = np.append(total_target, target.cpu().numpy())
                correct += pred.eq(target.view_as(pred)).sum().item()
            if(max_correct < correct):
                torch.save(model.state_dict(), MODEL_FILE)
                max_correct = correct
                print("Best accuracy! correct images: %5d"%correct)
        ema.resume(model)

        # Output                    
     
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100 * correct / len(test_loader.dataset)
        best_test_accuracy = 100 * max_correct / len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%) (best: {:.2f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), test_accuracy, best_test_accuracy))

        f = open(OUTPUT_FILE, 'a')
        f.write(" %3d %12.6f %9.3f %12.6f %9.3f %9.3f\n"%(epoch, train_loss, train_accuracy, test_loss, test_accuracy, best_test_accuracy))
        f.close()

        
        # mengupdate learning rate scheduler                                           
        lr_scheduler.step()
# seed, epoch, trial, gpu di input secara manual karena pada codingan sebelumnya diambil dari github sehingga tidak bisa dijalankan memakai code sebelumnya

#seed untuk menentukan nilai awal
p_seed = int(input ("Seeds: ")) # memasukkan angka sebagai input seed

#epoch digunakan untuk melakukan pengulangan (sesuai input) terhadap trial yang dijalankan
p_epoch = int(input ("Epoch: ")) # memasukkan angka sebagai input epoch

#trial digunakan untuk pengulangan percobaan
p_trials = int(input ("Trials: ")) # memasukkan angka sebagai input epoch

p_kernel_size = int (input ("Kernel size: ")) #memasukkan angka sebagai input epoch (disesuaikan dengan angka pada model)

#gpu untuk menjalankan program pada grafik card
p_gpu = int(input ("GPU: ")) #masukkan angka sebagai input GPU

p_logdir = input ("Logdir: ") #masukkan input model yang akan digunakan

#menjalankan GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(p_gpu)

for x in range (p_trials): #mengulang percobaan (trial)
  run(p_seed + x, p_epoch, p_kernel_size, p_logdir)


Seeds: 0
Epoch: 10
Trials: 10
Kernel size: 5
GPU: 0
Logdir: modelM5
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 24, 24]             800
       BatchNorm2d-2           [-1, 32, 24, 24]              64
            Conv2d-3           [-1, 64, 20, 20]          51,200
       BatchNorm2d-4           [-1, 64, 20, 20]             128
            Conv2d-5           [-1, 96, 16, 16]         153,600
       BatchNorm2d-6           [-1, 96, 16, 16]             192
            Conv2d-7          [-1, 128, 12, 12]         307,200
       BatchNorm2d-8          [-1, 128, 12, 12]             256
            Conv2d-9            [-1, 160, 8, 8]         512,000
      BatchNorm2d-10            [-1, 160, 8, 8]             320
           Linear-11                   [-1, 10]         102,400
      BatchNorm1d-12                   [-1, 10]              20
Total params: 1,128,180
Trainable p

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:992: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


Train Epoch: 0 [00000/60000 (0%)]	Loss: 2.835055
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.672545
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.446485
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.381814
Train Epoch: 0 [48000/60000 (80%)]	Loss: 0.333888
Best accuracy! correct images:  9892

Test set: Average loss: 0.1459, Accuracy: 9892/10000 (98.92%) (best: 98.92%)

Train Epoch: 1 [00000/60000 (0%)]	Loss: 0.322609
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.246928
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.257343
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.147037
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.212519

Test set: Average loss: 0.1495, Accuracy: 9817/10000 (98.17%) (best: 98.92%)

Train Epoch: 2 [00000/60000 (0%)]	Loss: 0.106649
Train Epoch: 2 [12000/60000 (20%)]	Loss: 0.104282
Train Epoch: 2 [24000/60000 (40%)]	Loss: 0.138825
Train Epoch: 2 [36000/60000 (60%)]	Loss: 0.153324
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.101405
Best accuracy! correct images:  9938

Test set: Average lo

TEST

In [ ]:
# mengimport libary
import sys
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import argparse
import numpy as np 
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from PIL import Image

#Running Proses Testing
def run(p_seed=0, p_kernel_size=5, p_logdir="temp"):

    # enable GPU usage 
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda == False:
        print("WARNING: CPU will be used for training.")
        exit(0)

    # data loader 
    test_dataset = MnistDataset(training=False, transform=None)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # model selection 
    if(p_kernel_size == 3):
        model1 = ModelM3().to(device)
    elif(p_kernel_size == 5):
        model1 = ModelM5().to(device)
    elif(p_kernel_size == 7):
        model1 = ModelM7().to(device)

    model1.load_state_dict(torch.load("/content/MnistSimpleCNN-master/logs/%s/model%03d.pth"%(p_logdir,p_seed))) #load data dari drive

    model1.eval()
    test_loss = 0
    correct = 0
    wrong_images = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model1(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            wrong_images.extend(np.nonzero(~pred.eq(target.view_as(pred)).cpu().numpy())[0]+(100*batch_idx))

    np.savetxt("/content/MnistSimpleCNN-master/logs/%s/wrong%03d.txt"%(p_logdir,p_seed), wrong_images, fmt="%d")
    print(len(wrong_images), wrong_images)

p_logdir = input ("Logdir: ") #masukkan input model yang akan digunakan
p_seed = int(input ("Seeds: ")) # input angka untuk seed
p_trials = int(input ("Trials: ")) # masukkan angka sebagi input berapa kali percobaan
p_kernel_size = int (input ("Kernel size: ")) # masukkan angka sebagai input ukuran kernel (menyesuaikan angka pada model)

for y in range (p_trials): #perulangan percobaan (trial)
  run(p_seed + y, p_kernel_size, p_logdir)

Logdir: modelM5
Seeds: 0
Trials: 10
Kernel size: 5
50 [193, 449, 582, 625, 938, 1226, 1232, 1403, 1901, 2035, 2040, 2130, 2135, 2293, 2326, 2447, 2462, 2488, 2534, 2597, 3073, 3225, 3534, 3558, 3762, 3906, 4018, 4065, 4201, 4740, 4761, 4823, 5457, 5955, 6558, 6576, 6597, 6783, 6883, 7216, 8316, 8325, 8387, 8408, 8527, 9009, 9024, 9729, 9754, 9850]
46 [449, 582, 625, 646, 659, 882, 938, 947, 1226, 1232, 1260, 1459, 1621, 1901, 2040, 2130, 2293, 2462, 2488, 2597, 3225, 3422, 3558, 3762, 3985, 4018, 4176, 4369, 4497, 4740, 4761, 4823, 4860, 5955, 6554, 6576, 6625, 7216, 8275, 8279, 8408, 8527, 9664, 9700, 9729, 9749]
54 [445, 447, 625, 938, 947, 1014, 1050, 1226, 1232, 1737, 1901, 2035, 2040, 2098, 2118, 2130, 2293, 2454, 2462, 2488, 2597, 2654, 2939, 2959, 3030, 3073, 3225, 3330, 3384, 3422, 3558, 3762, 3846, 3853, 4201, 4443, 4547, 4620, 4740, 4761, 4814, 4860, 5457, 5937, 6554, 6576, 6625, 7216, 8065, 8279, 8408, 8527, 9700, 9729]
52 [321, 445, 447, 582, 659, 938, 947, 1050, 1112, 1232

HOMO ENSEMBLE

In [ ]:
#mengimport libary
import numpy as np 
import argparse

cnt = 1
best = 10000
curr = 10000

p_kernel_size = int (input ("Kernel size: ")) #input ukuran kernel 

KERNEL_SIZE = p_kernel_size 

for i in range(10):
    for j in range(i+1,10):
        for k in range(j+1,10):
            w1 = np.loadtxt("/content/MnistSimpleCNN-master/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, i)).astype(np.int)
            w2 = np.loadtxt("/content/MnistSimpleCNN-master/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, j)).astype(np.int)
            w3 = np.loadtxt("/content/MnistSimpleCNN-master/logs/modelM%d/wrong%03d.txt"%(KERNEL_SIZE, k)).astype(np.int)

            board = np.zeros((10000))
            board[w1] += 1
            board[w2] += 1
            board[w3] += 1
            board = board // 2
            curr = np.sum(board)
            if curr < best:
                best = curr
            print("%4d %4d %4d %4d %4d %4d"%(cnt, len(w1), len(w2), len(w3), curr, best))
            cnt += 1


Kernel size: 5
   1   50   46   54   37   37
   2   50   46   52   38   37
   3   50   46   43   35   35
   4   50   46   50   41   35
   5   50   46   52   39   35
   6   50   46   44   35   35
   7   50   46   57   41   35
   8   50   46   35   36   35
   9   50   54   52   46   35
  10   50   54   43   45   35
  11   50   54   50   46   35
  12   50   54   52   41   35
  13   50   54   44   40   35
  14   50   54   57   47   35
  15   50   54   35   35   35
  16   50   52   43   38   35
  17   50   52   50   42   35
  18   50   52   52   38   35
  19   50   52   44   35   35
  20   50   52   57   45   35
  21   50   52   35   36   35
  22   50   43   50   40   35
  23   50   43   52   42   35
  24   50   43   44   37   35
  25   50   43   57   42   35
  26   50   43   35   37   35
  27   50   50   52   40   35
  28   50   50   44   36   35
  29   50   50   57   43   35
  30   50   50   35   34   34
  31   50   52   44   42   34
  32   50   52   57   44   34
  33   50   52   35   37 